In [ ]:
from pyspark import SparkConf, SparkContext
master = 'local[*]' # 指定local模式
appName = 'rdd'
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)

验证特性

In [3]:
# 查看分区排布
sc.parallelize(range(9), 3).glom().collect()

[[0, 1, 2], [3, 4, 5], [6, 7, 8]]

In [4]:
# 计算方法会作用到每个分片 split
print(sc.parallelize(range(9)).map(lambda x: x*10).collect())
print(sc.parallelize(range(9), 3).map(lambda x: x*10).collect())

[0, 10, 20, 30, 40, 50, 60, 70, 80]
[0, 10, 20, 30, 40, 50, 60, 70, 80]


In [5]:
# rdd对象的迭代依赖关系
rdd1 = sc.textFile('words.txt')
rdd2 = rdd1.flatMap(lambda x: x.split(' '))
rdd3 = rdd2.map(lambda x: (x, 1))
rdd4 = rdd3.reduceByKey(lambda a, b: a + b)
print(rdd4.collect())

[('hadoop', 2), ('', 2), ('zzj', 2), ('spark', 1)]


创建rdd

In [12]:
rdd = sc.parallelize(range(9)) # 本地集合变为分布式数据集合
# local[*]默认的分区数根据CPU核数的多线程来定，每个线程创建管理一个分区； lscpu CPU核数为16
rdd.getNumPartitions()  # 查看分区数
print(rdd.collect()) # 分布式数据集合变为本地集合

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [13]:
sc.parallelize(range(9), 3).getNumPartitions()

3

In [15]:
# 读取文件创建  默认分区数与CPU核数无关， 分区数参数太大了无效，spark会自己判断
rdd = sc.textFile("words.txt")
print(rdd.getNumPartitions()) 
print(sc.textFile("words.txt", 3).getNumPartitions())
print(sc.textFile("words.txt", 100).getNumPartitions())

2
4
29


In [18]:
rdd = sc.wholeTextFiles("./测试数据/tiny_files")
rdd.collect(), rdd.getNumPartitions()

([('file:/home/ubuntu/pyspark/测试数据/tiny_files/4.txt',
   'hello spark\r\nhello hadoop\r\nhello flink'),
  ('file:/home/ubuntu/pyspark/测试数据/tiny_files/2.txt',
   'hello spark\r\nhello hadoop\r\nhello flink'),
  ('file:/home/ubuntu/pyspark/测试数据/tiny_files/1.txt',
   'hello spark\r\nhello hadoop\r\nhello flink'),
  ('file:/home/ubuntu/pyspark/测试数据/tiny_files/3.txt',
   'hello spark\r\nhello hadoop\r\nhello flink'),
  ('file:/home/ubuntu/pyspark/测试数据/tiny_files/5.txt',
   'hello spark\r\nhello hadoop\r\nhello flink')],
 2)

In [17]:
rdd.map(lambda x: x[1]).collect()

['hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink']

# 

转换算子

In [19]:
rdd = sc.parallelize(["fnafnak nfalkfamalf afa", 'fankfsf fa faaf', 'wquiqw fsa'])
print(rdd.map(lambda x: x.split()).collect())
print(rdd.flatMap(lambda x: x.split()).collect())

[['fnafnak', 'nfalkfamalf', 'afa'], ['fankfsf', 'fa', 'faaf'], ['wquiqw', 'fsa']]
['fnafnak', 'nfalkfamalf', 'afa', 'fankfsf', 'fa', 'faaf', 'wquiqw', 'fsa']


In [20]:
rdd = sc.parallelize([('a', 1), ('a', 1), ('b', 2), ('b', 3), ('a', 1)])
rdd.reduceByKey(lambda a, b: a+b).collect()

[('b', 5), ('a', 3)]

In [21]:
print(rdd.mapValues(lambda x: x*10).collect())
print(rdd.map(lambda x: (x[0], x[1]*10)).collect())

[('a', 10), ('a', 10), ('b', 20), ('b', 30), ('a', 10)]
[('a', 10), ('a', 10), ('b', 20), ('b', 30), ('a', 10)]


In [34]:
print(rdd.groupBy(lambda x: x[0]).collect())
print(rdd.groupBy(lambda x: x[0]).map(lambda x: (x[0], list(x[1]))).collect())
print(rdd.groupByKey().map(lambda x: (x[0], list(x[1]))).collect())

[('b', <pyspark.resultiterable.ResultIterable object at 0x7f260b7ac760>), ('a', <pyspark.resultiterable.ResultIterable object at 0x7f260b7029a0>)]
[('b', [('b', 2), ('b', 3)]), ('a', [('a', 1), ('a', 1), ('a', 1)])]
[('b', [2, 3]), ('a', [1, 1, 1])]


In [25]:
rdd.filter(lambda x: x[1]%2==0).collect()

[('b', 2)]

In [26]:
print(sc.parallelize([1, 1, 1, 2, 2]).distinct().collect())
print(rdd.distinct().collect())

[1, 2]
[('b', 2), ('a', 1), ('b', 3)]


In [27]:
rdd.union(sc.parallelize([1,2])).collect()

[('a', 1), ('a', 1), ('b', 2), ('b', 3), ('a', 1), 1, 2]

In [32]:
rdd_person = sc.parallelize([(1001, "zzj"), (1002, 'ymm'), (1003, 'xxx')])
rdd_depart = sc.parallelize([(1001, 'dep1'), (1002, "dep2")])
print(rdd_person.join(rdd_depart).collect())
print(rdd_person.leftOuterJoin(rdd_depart).collect())

[(1001, ('zzj', 'dep1')), (1002, ('ymm', 'dep2'))]
[(1001, ('zzj', 'dep1')), (1002, ('ymm', 'dep2')), (1003, ('xxx', None))]


In [33]:
rdd2 = sc.parallelize(range(9), 2).glom() # 嵌套
print(rdd2.collect()) 
print(rdd2.flatMap(lambda x: x).collect()) # 解嵌套

[[0, 1, 2, 3], [4, 5, 6, 7, 8]]
[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [37]:
print(rdd.collect())
rdd.sortBy(lambda x: x[1], ascending=True, numPartitions=10).collect()

[('a', 1), ('a', 1), ('b', 2), ('b', 3), ('a', 1)]


[('a', 1), ('a', 1), ('a', 1), ('b', 2), ('b', 3)]

In [38]:
sc.parallelize([('a', 1), ('A', 2), ('b', 1)]).sortByKey(True, 3, lambda x: x.lower()).collect()

[('a', 1), ('A', 2), ('b', 1)]

案例：返回在北京购买了哪些类别的商品 

In [4]:
file_rdd = sc.textFile("测试数据/order.text")
json_rdd = file_rdd.flatMap(lambda x: x.split("|"))
json_rdd.collect()

['{"id":1,"timestamp":"2019-05-08T01:03.00Z","category":"平板电脑","areaName":"北京","money":"1450"}',
 '{"id":2,"timestamp":"2019-05-08T01:01.00Z","category":"手机","areaName":"北京","money":"1450"}',
 '{"id":3,"timestamp":"2019-05-08T01:03.00Z","category":"手机","areaName":"北京","money":"8412"}',
 '{"id":4,"timestamp":"2019-05-08T05:01.00Z","category":"电脑","areaName":"上海","money":"1513"}',
 '{"id":5,"timestamp":"2019-05-08T01:03.00Z","category":"家电","areaName":"北京","money":"1550"}',
 '{"id":6,"timestamp":"2019-05-08T01:01.00Z","category":"电脑","areaName":"杭州","money":"1550"}',
 '{"id":7,"timestamp":"2019-05-08T01:03.00Z","category":"电脑","areaName":"北京","money":"5611"}',
 '{"id":8,"timestamp":"2019-05-08T03:01.00Z","category":"家电","areaName":"北京","money":"4410"}',
 '{"id":9,"timestamp":"2019-05-08T01:03.00Z","category":"家具","areaName":"郑州","money":"1120"}',
 '{"id":10,"timestamp":"2019-05-08T01:01.00Z","category":"家具","areaName":"北京","money":"6661"}',
 '{"id":11,"timestamp":"2019-05-08T05:03.00Z","

In [6]:
import json
dict_rdd = json_rdd.map(lambda x: json.loads(x))
dict_rdd.collect()

[{'id': 1,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '平板电脑',
  'areaName': '北京',
  'money': '1450'},
 {'id': 2,
  'timestamp': '2019-05-08T01:01.00Z',
  'category': '手机',
  'areaName': '北京',
  'money': '1450'},
 {'id': 3,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '手机',
  'areaName': '北京',
  'money': '8412'},
 {'id': 4,
  'timestamp': '2019-05-08T05:01.00Z',
  'category': '电脑',
  'areaName': '上海',
  'money': '1513'},
 {'id': 5,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '家电',
  'areaName': '北京',
  'money': '1550'},
 {'id': 6,
  'timestamp': '2019-05-08T01:01.00Z',
  'category': '电脑',
  'areaName': '杭州',
  'money': '1550'},
 {'id': 7,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '电脑',
  'areaName': '北京',
  'money': '5611'},
 {'id': 8,
  'timestamp': '2019-05-08T03:01.00Z',
  'category': '家电',
  'areaName': '北京',
  'money': '4410'},
 {'id': 9,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '家具',
  'areaName': '郑州',
  'money': '1120'},
 {'id': 

In [7]:
beijing_rdd = dict_rdd.filter(lambda x: x['areaName']=="北京")
beijing_rdd.collect()

[{'id': 1,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '平板电脑',
  'areaName': '北京',
  'money': '1450'},
 {'id': 2,
  'timestamp': '2019-05-08T01:01.00Z',
  'category': '手机',
  'areaName': '北京',
  'money': '1450'},
 {'id': 3,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '手机',
  'areaName': '北京',
  'money': '8412'},
 {'id': 5,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '家电',
  'areaName': '北京',
  'money': '1550'},
 {'id': 7,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '电脑',
  'areaName': '北京',
  'money': '5611'},
 {'id': 8,
  'timestamp': '2019-05-08T03:01.00Z',
  'category': '家电',
  'areaName': '北京',
  'money': '4410'},
 {'id': 10,
  'timestamp': '2019-05-08T01:01.00Z',
  'category': '家具',
  'areaName': '北京',
  'money': '6661'},
 {'id': 12,
  'timestamp': '2019-05-08T01:01.00Z',
  'category': '书籍',
  'areaName': '北京',
  'money': '5550'},
 {'id': 13,
  'timestamp': '2019-05-08T01:03.00Z',
  'category': '书籍',
  'areaName': '北京',
  'money': '5550'},
 {'id

In [9]:
cat_rdd = beijing_rdd.map(lambda x: f'北京_{x["category"]}').distinct()
cat_rdd.collect()

['北京_平板电脑', '北京_家具', '北京_书籍', '北京_食品', '北京_服饰', '北京_手机', '北京_家电', '北京_电脑']

动作算子

In [11]:
rdd_file = sc.textFile("测试数据/words.txt")
rdd_words = rdd_file.flatMap(lambda x: x.split()).map(lambda x: (x, 1))
rdd_words.countByKey(), rdd_words.reduceByKey(lambda a, b: a+b).collect()

(defaultdict(int, {'hello': 3, 'spark': 1, 'hadoop': 1, 'flink': 1}),
 [('hadoop', 1), ('hello', 3), ('spark', 1), ('flink', 1)])

In [12]:
sc.parallelize(range(9)).reduce(lambda a,b: a+b)

36

In [21]:
rdd_ = sc.parallelize([1, 9, 2, 8], 2)
print(rdd_.glom().collect())
rdd_.fold(10, lambda a, b: a+b)

[[1, 9], [2, 8]]


50

In [27]:
sc.parallelize(range(8)).first(), sc.parallelize(range(8)).take(5), sc.parallelize(range(8)).top(5)

(0, [0, 1, 2, 3, 4], [7, 6, 5, 4, 3])

In [30]:
sc.parallelize(range(8)).count(), sc.parallelize(range(8), 2).count(), sc.parallelize([(1, 2), (1, 2)]).count()

(8, 8, 2)

In [31]:
sc.parallelize(range(8)).takeSample(True, 22), sc.parallelize(range(8)).takeSample(False, 22)

([0, 2, 0, 5, 4, 0, 2, 7, 0, 3, 3, 1, 0, 4, 6, 4, 2, 3, 6, 2, 0, 4],
 [1, 2, 5, 7, 4, 3, 0, 6])

In [32]:
sc.parallelize(range(8)).takeSample(False, 2, 1), sc.parallelize(range(8)).takeSample(False, 2, 1)

([3, 6], [3, 6])

In [33]:
sc.parallelize([1,2,4,1,42,2]).takeOrdered(3, lambda x: -x)

[42, 4, 2]

In [36]:
rdd_num = sc.parallelize(range(8))
rdd_num.foreach(lambda x: x*10), rdd_num.collect(), rdd_num.foreach(lambda x: print(x*10))

0
40
20
60
30
50
70
10


(None, [0, 1, 2, 3, 4, 5, 6, 7], None)

In [40]:
rdd_num.getNumPartitions(), rdd_num.saveAsTextFile("测试数据/save")

(16, None)

In [41]:
sc.parallelize(range(8), 3).saveAsTextFile("测试数据/save2")

分区转换算子

In [42]:
rdd_parts = sc.parallelize(range(8), 3)
def process(part):
    return [i*10 for i in part]
     
rdd_parts.mapPartitions(process).collect()

[0, 10, 20, 30, 40, 50, 60, 70]

In [46]:
def process2(part):
    res = [i*10 for i in part]
    print(res)
    return res
     
rdd_parts.collect(), rdd_parts.foreachPartition(process2)

[50, 60, 70]
[20, 30, 40]
[0, 10]


([0, 1, 2, 3, 4, 5, 6, 7], None)

In [51]:
rdd_kv = sc.parallelize([(1, 2), (2, 3), (3, 2), (1, 3), (2, 3), (34,3)])
def custom_parts(K):
    return K%3
rdd_kv.collect(), rdd_kv.partitionBy(3, custom_parts).glom().collect()

([(1, 2), (2, 3), (3, 2), (1, 3), (2, 3), (34, 3)],
 [[(3, 2)], [(1, 2), (1, 3), (34, 3)], [(2, 3), (2, 3)]])

In [53]:
rdd_num.getNumPartitions(), rdd_num.repartition(3).getNumPartitions()

(16, 3)

In [55]:
rdd_num.coalesce(1).getNumPartitions(), rdd_num.coalesce(17).getNumPartitions(), rdd_num.coalesce(17, True).getNumPartitions()

(1, 16, 17)

In [1]:
from pyspark import SparkConf, SparkContext, StorageLevel
import jieba 

appname = 'search_words'
master = 'local[*]'
conf = SparkConf().setAppName(appname).setMaster(master)
sc = SparkContext(conf=conf)

rdd_file = sc.textFile('测试数据/SogouQ.txt')
print(rdd_file.takeSample(False, 3))


23/10/10 12:58:55 WARN Utils: Your hostname, 10-24-17-107 resolves to a loopback address: 127.0.1.1; using 10.24.17.107 instead (on interface eth0)
23/10/10 12:58:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/10 12:58:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


['04:00:22\t7044693659960919\t数据仓库\t10\t8\thttp://www.itcast.cn', '20:00:07\t0014362172758659586\thadoop\t64\t21\thttp://www.itcast.cn', '14:00:09\t19701804785360144\t传智专修学院\t1\t1\thttp://www.itcast.cn']
